### I have embarked on a project to explore the difference between investing in ETFs vs individual stocks

The goal of this project is to eventually create an application that houses all of an individual's investment accounts, with the ability to compare against benchmarks, and also visualize predicted gains for retirement, showing the importance of investing early and often.


In [1]:
# Importing initial libraries.
import pandas as pd
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import bs4
import requests
from bs4 import BeautifulSoup
from pandas_datareader import data, wb
%matplotlib inline

I wanted to explore the idea of getting data via webscraping, since both the Yahoo Finance API and the Google Finance API were deprecated 

### Webscraping for Ticker Info

#### First I will use a single security, VTI in this instance, to make sure I am pulling the data correctly

In [2]:
r = requests.get('https://finance.yahoo.com/quote/VTI?p=VTI')

In [3]:
soup = bs4.BeautifulSoup(r.text, "lxml") 
#I originally had this as XML, but it broke for some reason.  A switch to LXML quickly fixed this.

In [4]:
#using the inspect aspect of a webpage, this is simply parsong a little HTML.  
soup.find_all(name = 'div', attrs = {'class':'My(6px) Pos(r) smartphone_Mt(6px)'})[0].find('span').text

'151.97'

### Creating a Function
#### Here I create a function that allows you to pass any ticker symbol into it and retrieve the current price from yahoo finance's webpage

In [5]:
def parseCurrentPrice(stock):
    r = requests.get('https://finance.yahoo.com/quote/{}?p={}'.format(stock,stock))
    soup = bs4.BeautifulSoup(r.text, "lxml")
    price = soup.find_all(name = 'div', attrs = {'class':'My(6px) Pos(r) smartphone_Mt(6px)'})[0].find('span').text
    return print('The current price of ' + str(stock).upper() + ' is: ' + str(price))

In [6]:
parseCurrentPrice('fb')
#boom! 

The current price of FB is: 188.89


#### I will now try to capture historic data for tickers

In [7]:
#create an empty list to add data to.
date_list=[]
high_list=[]
adjclose_list=[]

In [8]:
r2 = requests.get('https://finance.yahoo.com/quote/VTI/history?p=VTI')
soup2 = bs4.BeautifulSoup(r2.text, "lxml")
#below I was exploring the ability to change the intervals displayed on the page to show more days
#https://finance.yahoo.com/quote/VTI/history?period1=992588400&period2=1570258800&interval=1d&filter=history&frequency=1d

In [9]:
soup2.find(name = 'tr', attrs = {'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'}) \
     .find(name = 'td', attrs = {'class': 'Py(10px) Ta(start) Pend(10px)'}).find('span').text

'Oct 15, 2019'

In [10]:
tr = soup2.find_all(name = 'tr', attrs = {'class':'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)'})
#the above code sets the variable tr to the HTML element that holds every days' information.  Use find_all to capture every day
#the below function will iterate through every day and pull elements out related to the date
for row in tr:
    date_list.append(row.find(name = 'td', attrs = {'class': 'Py(10px) Ta(start) Pend(10px)'}).find('span').text)

In [11]:
#A quick check of the list shows we have successfully filled date_list with the dates corresponding to the market being open
date_list[0:5]

['Oct 15, 2019',
 'Oct 14, 2019',
 'Oct 11, 2019',
 'Oct 10, 2019',
 'Oct 09, 2019']

At this point, I tried to run the same function iterating through every day to collect data on the high and adjusted close of each day by adding the following lines to my `for row in tr` function:
- high_list.append(row.find_all(name = 'td')[0].text)
- adjclose_list.append(row.find_all(name = 'td')[4].text)

but I kept running into an error stating the list index out of range.  I realized the issue is with the dividends.  Whenever there was a dividend issued, there would not be highs, lows, and closing values.

I tried creating logic that would remove the dividends by finding the specific HTML elements that represented a dividend, and adding logic to only add the days where that element was empty, but got stuck.

The following code represents a dividend day:

```date_list.append(row.find_all(name = 'td', attrs = {'class': 'Ta(c) Py(10px) Pstart(10px)'}))```

      if row.find_all(name = 'td', attrs = {'class': 'Ta(c) Py(10px) Pstart(10px)'}) != '' :
        date_list.append(row.find_all(name = 'td', attrs = {'class': 'Ta(c) Py(10px) Pstart(10px)'}))
        high_list.append(row.find_all(name = 'td')[0].text)
        adjclose_list.append(row.find_all(name = 'td')[4].text)

#### At this point, I started to explore beyond Yahoo Finance for historical data, so see if another site's HTML was easier to parse

In [19]:
date_list2 = []
r3 = requests.get('https://seekingalpha.com/symbol/VTI/historical-price-quotes')
soup3 = bs4.BeautifulSoup(r3.text, "lxml")

In [21]:
soup3.find('table').find_all('tr')
# historical-quotes-table

[<tr><th class="quote-date" data-sort="Date" data-type="date">Date</th><th class="quote-open" data-sort="Open">Open</th><th class="quote-high" data-sort="High">High</th><th class="quote-low" data-sort="Low">Low</th><th class="quote-close" data-sort="Close">Close</th><th class="quote-volume" data-sort="Volume">Volume</th><th class="quote-change" data-sort="PercentChange">Change %</th></tr>]

In [22]:
tr2 = soup3.find('table').find('tr').find('th').text
tr2

'Date'

#### We will come back to that attempt.

### I will now attempt to have my program download a csv and read it for historical data

In [23]:
from selenium import webdriver

In [26]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://finance.yahoo.com/quote/VTI/history?p=VTI')


Checking for win32 chromedriver:77.0.3865.40 in cache
Driver found in C:\Users\ryanJ\.wdm\chromedriver\77.0.3865.40\win32/chromedriver.exe


In [27]:
download_data = driver.find_element_by_link_text('Download Data')
#download data is the HTML name of the button

download_data.click()
#click on the download data link

In [28]:
#downloaded it to the directory, now reading it from the CSV on my drive.
df = pd.read_csv('VTI.csv', index_col='Date')
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-09-19,150.119995,150.479996,150.009995,150.149994,146.546326,1809800
2018-09-20,150.830002,151.470001,150.520004,151.309998,147.678497,1713900
2018-09-21,151.820007,151.839996,151.050003,151.160004,147.532104,1986100
2018-09-24,150.869995,150.929993,150.270004,150.610001,146.995300,1668600
2018-09-25,150.889999,151.070007,150.419998,150.529999,146.917206,2318900


I realized I need to write a function to have it overwrite the CSV, or have create a variable that increments.

In [29]:
def incrementer(): 
    csv = str(VTI.csv(x))


I will come back to this for practice, but will continue to search for better options for getting historical data.

In the meantime, I continued with selenium to find the elements by the HTML path

In [41]:
driver.find_element_by_xpath('// tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]').text
#Xpath=//tagname[@attribute='value']

'Oct 15, 2019 151.03 152.33 150.91 151.97 151.97 2,152,724'

In [40]:
driver.find_element_by_xpath('// tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]/td').text

'Oct 15, 2019'

In [58]:
driver.find_element_by_xpath('// table[@class="W(100%) M(0)"]') \
      .find_element_by_xpath('// tr[@class="BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)"]') \
      .find_elements_by_xpath('// td')[4].text

'151.97'